In [3]:
import os
import azureml

from azureml.train.estimator import Estimator
from azureml.train.dnn import PyTorch
from azureml.core import Workspace, Datastore, Experiment, Model, Run
from azureml.core.compute import ComputeTarget
from azureml.widgets import RunDetails

from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.core.graph import PipelineParameter
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
from azureml.data.datapath import DataPath, DataPathComputeBinding
from azureml.data.data_reference import DataReference

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.85


# Connect to environment

In [4]:
# Connect to workspace
ws = Workspace.from_config()
print("Workspace:",ws.name,"in region", ws.location)

# Connect to compute cluster
cluster = ComputeTarget(workspace=ws, name="OptimusPrime")
print('Compute cluster:', cluster.name)

# Connect to the default datastore
ds = ws.get_default_datastore()
print("Datastore:",ds.name)

# Connect to the experiment
experiment = Experiment(workspace=ws, name='seer-from-notebook')
print("Experiment:",experiment.name)

Workspace: Cybertron in region westeurope
Compute cluster: OptimusPrime
Datastore: workspaceblobstore
Experiment: seer-from-notebook


# Creating the Pipeline

In [ ]:
# Parameters make it easy for us to re-run this training pipeline, including for retraining.
source_dataset = DataPath(
    datastore=ds, 
    path_on_datastore="seer")

source_dataset_param = (PipelineParameter(name="source_dataset",default_value=source_dataset),
                          DataPathComputeBinding())

# Location for the step scripts
script_folder = "./scripts"

# Name of the model
model_name = "seer"

# Experiment name
experiment_name = "seer-from-notebook"

## Step 1 - Data preparation

In [10]:
# Output location for the pre-proccessed trainings images
training_data_location = PipelineData(name="seertrainingdata", datastore=ds)

# Create the pre-process step
preProcessDataStep = PythonScriptStep(name="Pre-process data",
                            script_name="steps/prep.py",
                            compute_target=cluster,
                            inputs=[source_dataset_param],
                            arguments=['--source_path', source_dataset_param,
                                       '--destination_path', training_data_location
                                      ],
                            outputs=[training_data_location],
                            source_directory=script_folder)

## Step 2 - Train the model

In [11]:
# Output location for the produced model
model_location = PipelineData(name="model", datastore=ds, output_path_on_compute="model")

# Estimator script params
estimator_script_params = [
    "--data-folder", training_data_location,
    "--output-folder", model_location
]

# Create the tensorflow Estimator
trainEstimator = PyTorch(
                     source_directory = script_folder,
                     compute_target = cluster,
                     entry_script = "steps/train.py", 
                     use_gpu = True,
                     framework_version='1.3'
                )

# Create a pipeline step with the TensorFlow Estimator
trainOnGpuStep = EstimatorStep(
    name='Train Estimator Step',
    estimator=trainEstimator,
    inputs=[training_data_location],
    outputs=[model_location],
    compute_target=cluster,
    estimator_entry_script_arguments = estimator_script_params
) 


## Step 3 - Register the model

In [12]:
registerModelStep = PythonScriptStep(name="Register model in Model Management",
                            script_name="steps/register.py",
                            compute_target=cluster,
                            inputs=[model],
                            arguments=['--model_name', model_name,
                                       '--model_assets_path', model_location
                                      ],
                            source_directory=script_folder)

## Create the pipeline

In [13]:
seer_pipeline = Pipeline(workspace=ws, steps=[preProcessDataStep,trainOnGpuStep,registerModelStep])
seer_pipeline.validate()

Step Pre-process data is ready to be created [40b7e028]
Step Train Estimator Step is ready to be created [f79f9259]
Step Register model in Model Management is ready to be created [6a578a72]
Data reference workspaceblobstore_42582266 is ready to be created [34214cdb], (Consumers of this data will generate new runs.)


[]

In [14]:
mlpipeline = seer_pipeline.publish(name="Seer - Training pipeline (From Notebook)",)

print("Pipeline Published ID:"+mlpipeline.id)

Created step Pre-process data [40b7e028][7a391c3d-e1f8-492d-b190-51d0101a15d0], (This step will run and generate new outputs)
Created step Train Estimator Step [f79f9259][9ec22dab-0bea-4cb8-a4a2-0473092a0537], (This step will run and generate new outputs)
Created step Register model in Model Management [6a578a72][4eb74218-8992-4bec-ba6f-014cf09b89f1], (This step will run and generate new outputs)
Created data reference workspaceblobstore_42582266 for StepId [34214cdb][c4002a67-2f12-423a-b06b-31793d88d71d], (Consumers of this data will generate new runs.)
Pipeline Published ID:8e8222ea-0ed0-4b9c-b049-8da14e645321


In [15]:
pipeline_run = mlpipeline.submit(ws,experiment_name)

Submitted PipelineRun 37b657ea-e37c-4d73-be1b-cd18649695e0
Link to Azure Machine Learning studio: https://ml.azure.com/experiments/seer-from-notebook/runs/37b657ea-e37c-4d73-be1b-cd18649695e0?wsid=/subscriptions/431dbae5-40ca-438a-8daf-77d7d5580b41/resourcegroups/Cybertron-RG/workspaces/Cybertron


In [16]:
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [ ]:
pipeline_run.cancel()

In [17]:
oldrun = [r for r in experiment.get_runs() if r.id == '6b60360a-89f9-41d7-98db-fd4becf143af'][0]

In [18]:
RunDetails(oldrun).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…